**Занятие первое**

Начнем с простого. Многие знают что такое map и reduce операции, но все же для закрпеления мы их тут реализуем. Ах да, не забудем и про shuffle. Делать все будем на упрощенной задаче с word count для ознакомления с самим подходом.

На самом деле мы рассмптрим все в упрощенном виде, но это даст нам понимание, как можно через hadoop streaming, например, писать самописные map и reduce операции

! mapred streaming \
  -input /wiki/sample.jsonl \
  -output /word-count \
  -mapper "/opt/conda/bin/python3.6 mapper.py" \
  -reducer "/opt/conda/bin/python3.6 reducer.py" \
  -file mapper.py \
  -file reducer.py

Выше mapper.py и reducer.py это программы, которые выполняют одноименные операции нам потоком информации из jsonl файла, записывая ответ в файл word-count

In [1]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
import re 
import string

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\slaza\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\slaza\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Давайте загрузим файл с текстом и посмотрим на него

In [2]:
with open('spark_text.txt', 'rb') as f:
    data = f.readlines()
data = [text.decode() for text in data if text.decode() != '\r\n']    

In [3]:
len(data)

60

In [4]:
data[1]

'Apache Spark has its architectural foundation in the resilient distributed dataset (RDD), a read-only multiset of data items distributed over a cluster of machines, that is maintained in a fault-tolerant way.[2] The Dataframe API was released as an abstraction on top of the RDD, followed by the Dataset API. In Spark 1.x, the RDD was the primary application programming interface (API), but as of Spark 2.x use of the Dataset API is encouraged[3] even though the RDD API is not deprecated.[4][5] The RDD technology still underlies the Dataset API.[6][7]\r\n'

Как бы мы сделали..
Надо немного почистить слова, а также сделать все в парадигме MapReduce. Понятно, что можно все написать проще, но мы ведь хотим понять, как это работает=)

Загрузим стоп слова, очистим от них текст, приведем к нижнему регистру, всем раздадим ключи

In [5]:
stop_words = stopwords.words("english")
stop_words = set(stop_words)

In [6]:
stop_words

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

пунктуацию тоже полезно бы удалить

In [7]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [8]:
def mapper_text(text):
    clean_text = re.sub(rf"[{string.punctuation}]", "", text)
    words = nltk.word_tokenize(clean_text)
    words_with_value = [(word.lower(), 1) for word in words 
                        if word not in stop_words]
    words_with_value = sorted(words_with_value, key=lambda x:x[0])
    return words_with_value

def create_chunks(shuffled_data):
    result = {}
    for idx, data in shuffled_data:
        if idx in result:
            result[idx].append(data)
        else:
            result[idx] = [data]
    return list(result.items())

def shuffle_text(mapper_result, n_nodes=5):
    shuffled_data = []
    for key, value in mapper_result:
        shuffled_data.append((hash(key)%n_nodes, (key, value)))
    shuffled_data = sorted(shuffled_data, key=lambda x: x[0])
    chunks = create_chunks(shuffled_data)
    return chunks


# на самом деле для reduce в жизни пишут иначе..не зря мы сортируем внутри map
#данные по ключам. Это нужно для избавления от этапа проверки ключа и поиска
def reduce_text(values_to_reduce):
    result = {}
    for key, value in values_to_reduce:
        if key in result:
            result[key] += 1
        else:
            result[key] = 1
    return result

Проверим, что все работает

Сначала map

In [9]:
data[0]

'Apache Spark is an open-source unified analytics engine for large-scale data processing. Spark provides an interface for programming entire clusters with implicit data parallelism and fault tolerance. Originally developed at the University of California, Berkeley AMPLab, the Spark codebase was later donated to the Apache Software Foundation, which has maintained it since.\r\n'

In [10]:
map_stage = mapper_text(data[0])

In [11]:
map_stage

[('amplab', 1),
 ('analytics', 1),
 ('apache', 1),
 ('apache', 1),
 ('berkeley', 1),
 ('california', 1),
 ('clusters', 1),
 ('codebase', 1),
 ('data', 1),
 ('data', 1),
 ('developed', 1),
 ('donated', 1),
 ('engine', 1),
 ('entire', 1),
 ('fault', 1),
 ('foundation', 1),
 ('implicit', 1),
 ('interface', 1),
 ('largescale', 1),
 ('later', 1),
 ('maintained', 1),
 ('opensource', 1),
 ('originally', 1),
 ('parallelism', 1),
 ('processing', 1),
 ('programming', 1),
 ('provides', 1),
 ('since', 1),
 ('software', 1),
 ('spark', 1),
 ('spark', 1),
 ('spark', 1),
 ('tolerance', 1),
 ('unified', 1),
 ('university', 1)]

shuffle

In [12]:
shuffle_stage = shuffle_text(map_stage, 5)

In [13]:
shuffle_stage

[(0,
  [('berkeley', 1),
   ('engine', 1),
   ('fault', 1),
   ('later', 1),
   ('parallelism', 1)]),
 (1,
  [('clusters', 1),
   ('codebase', 1),
   ('data', 1),
   ('data', 1),
   ('maintained', 1),
   ('originally', 1),
   ('processing', 1),
   ('programming', 1),
   ('software', 1),
   ('tolerance', 1)]),
 (2,
  [('analytics', 1),
   ('california', 1),
   ('implicit', 1),
   ('interface', 1),
   ('largescale', 1),
   ('unified', 1)]),
 (3,
  [('developed', 1),
   ('donated', 1),
   ('entire', 1),
   ('provides', 1),
   ('since', 1),
   ('university', 1)]),
 (4,
  [('amplab', 1),
   ('apache', 1),
   ('apache', 1),
   ('foundation', 1),
   ('opensource', 1),
   ('spark', 1),
   ('spark', 1),
   ('spark', 1)])]

reduce

In [14]:
reduce_text(shuffle_stage[4][1])

{'amplab': 1, 'apache': 2, 'foundation': 1, 'opensource': 1, 'spark': 3}

Итак, осталось все рассчитать параллельно и собрать результаты

In [15]:
from joblib import Parallel, delayed

In [16]:
n_nodes = 5

Обернем в 1 функциию для удобства map и shuffle

In [17]:
def map_shuffle(text, n_nodes):
    map_result = mapper_text(text)
    shuffle_result = shuffle_text(map_result, n_nodes)
    return shuffle_result

In [18]:
with Parallel(n_jobs=n_nodes, verbose=10, batch_size=5) as parallel:
    res = parallel(delayed(map_shuffle)(df, n_nodes) for df in data)

[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   6 tasks      | elapsed:    0.5s
[Parallel(n_jobs=5)]: Done  25 tasks      | elapsed:    0.5s
[Parallel(n_jobs=5)]: Done  60 out of  60 | elapsed:    0.5s finished


In [19]:
len(res)

60

In [20]:
res[0]

[(0,
  [('california', 1),
   ('codebase', 1),
   ('foundation', 1),
   ('largescale', 1),
   ('originally', 1),
   ('processing', 1),
   ('programming', 1),
   ('software', 1),
   ('spark', 1),
   ('spark', 1),
   ('spark', 1),
   ('unified', 1)]),
 (1,
  [('berkeley', 1),
   ('clusters', 1),
   ('donated', 1),
   ('maintained', 1),
   ('opensource', 1),
   ('provides', 1)]),
 (2,
  [('amplab', 1),
   ('analytics', 1),
   ('developed', 1),
   ('engine', 1),
   ('fault', 1),
   ('parallelism', 1),
   ('tolerance', 1),
   ('university', 1)]),
 (3, [('apache', 1), ('apache', 1), ('data', 1), ('data', 1), ('since', 1)]),
 (4, [('entire', 1), ('implicit', 1), ('interface', 1), ('later', 1)])]

Сделаем что-то вроде перессылки, собирая все в словари и заодно посмотрим на сколько равномерно распределлиись наши слова

In [21]:
shuffle_stage = {i:[] for i in range(5)}
for values in res:
    values = dict(values)
    for key in values.keys():
        shuffle_stage[key].extend(values[key])

In [22]:
for key in shuffle_stage.keys():
    print(f'{key}: number of words = {len(shuffle_stage[key])}')

0: number of words = 430
1: number of words = 368
2: number of words = 307
3: number of words = 377
4: number of words = 439


И последний этап - нужно сделать reduce

In [23]:
with Parallel(n_jobs=n_nodes, verbose=10, batch_size=5) as parallel:
    res = parallel(delayed(reduce_text)(shuffle_stage[key]) for key in shuffle_stage.keys())

[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   2 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=5)]: Done   3 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed:    0.0s finished


In [24]:
len(res)

5

In [25]:
res[0]

{'california': 1,
 'codebase': 1,
 'foundation': 3,
 'largescale': 1,
 'originally': 2,
 'processing': 12,
 'programming': 8,
 'software': 2,
 'spark': 44,
 'unified': 1,
 '1x': 1,
 'dataframe': 1,
 'distributed': 9,
 'faulttolerant': 1,
 'followed': 1,
 'still': 2,
 'the': 2,
 'top': 3,
 'use': 4,
 '2012': 1,
 'across': 4,
 'programs': 3,
 'reduce': 3,
 'restricted': 3,
 'implementation': 2,
 'interactiveexploratory': 1,
 'latency': 1,
 'ms': 1,
 'querying': 1,
 'class': 1,
 'formed': 1,
 'implementation29': 1,
 'iterative': 1,
 'standalone': 1,
 'supports': 3,
 'also': 9,
 'cassandra14': 1,
 'core': 6,
 'development': 1,
 'file': 4,
 'for': 1,
 'hadoop': 4,
 'hdfs12': 1,
 'machine': 1,
 'mode': 1,
 'required': 3,
 'system': 4,
 'usually': 1,
 'wide': 1,
 'centered': 1,
 'connect': 1,
 'net16': 1,
 'nonjvm': 1,
 'overall': 1,
 'scheduling': 3,
 'task': 1,
 'contain': 1,
 'filter': 1,
 'these': 3,
 'track': 1,
 'forms': 1,
 'imperative': 1,
 'rddoriented': 1,
 'flatmap': 1,
 'item': 1,

Собираем результат

In [26]:
result = {}
for partition in res:
    for key in partition.keys():
        if key in result:
            result[key] += partition[key]
        else:
            result[key] = partition[key]

In [27]:
sorted(result.items(), key=lambda x: x[1], reverse=True)

[('spark', 110),
 ('apache', 56),
 ('data', 44),
 ('streaming', 36),
 ('distributed', 23),
 ('processing', 19),
 ('sql', 17),
 ('also', 15),
 ('rdd', 15),
 ('learning', 15),
 ('api', 14),
 ('structured', 13),
 ('machine', 12),
 ('cluster', 12),
 ('hadoop', 11),
 ('mapreduce', 10),
 ('interface', 10),
 ('provides', 10),
 ('rdds', 10),
 ('programming', 9),
 ('the', 9),
 ('use', 9),
 ('core', 9),
 ('mllib', 9),
 ('framework', 9),
 ('graph', 9),
 ('support', 8),
 ('used', 8),
 ('tasks', 8),
 ('code', 8),
 ('python', 8),
 ('application', 8),
 ('run', 7),
 ('it', 7),
 ('dataset', 7),
 ('deep', 7),
 ('in', 7),
 ('operations', 7),
 ('algorithms', 7),
 ('graphx', 7),
 ('implementation', 6),
 ('pipelines', 6),
 ('two', 6),
 ('scala', 6),
 ('developers', 6),
 ('including', 6),
 ('map', 6),
 ('abstraction', 6),
 ('big', 6),
 ('simple', 6),
 ('batch', 6),
 ('graphs', 6),
 ('top', 5),
 ('across', 5),
 ('supports', 5),
 ('system', 5),
 ('these', 5),
 ('parallel', 5),
 ('project', 5),
 ('applications'

Да, было бы проще все сделать иным кодом и в один проход, но целью было разобрать, как все это примерно работает под капотом на больших данных.

**Домашнее задание**

Посчитать количество рейтингов больше 4 для каждого фильма и вывести фильмы в порядке убывания количества этих оценок

In [28]:
with open('user_ratedmovies.dat', 'rb') as f:
    data = f.readlines()
headers = data[0].decode().split('\t')[:3]
data = [row.decode().split('\t')[:3] for row in data[1:]]

In [29]:
headers

['userID', 'movieID', 'rating']

In [30]:
data[0]

['75', '3', '1']

In [31]:
len(data)

855598

Пишем map, shiffle и reduce + параллелим вычисления. Лучше задавать batch_size при распараллеливании, либо даже заранее все разбить на батчи, будет быстрее

Также посмотрите на то, нет ли перекоса в данных после shuffle, можете попробовать использовать остаток от деления не простого hash, а ввести какую-то функию

In [32]:
def map_rating(row):
    _, movie_id, movie_rating = row
    if float(movie_rating) > 4:
        return [(movie_id, 1)]
    else:
        return []

def create_chunks(shuffled_data):
    result = {}
    for idx, data in shuffled_data:
        if idx in result:
            result[idx].append(data)
        else:
            result[idx] = [data]
    return list(result.items())

def shuffle_rating(mapper_result, n_nodes=5):
    shuffled_data = []
    for key, value in mapper_result:
        shuffled_data.append((hash(key) % n_nodes, (key, value)))
    shuffled_data = sorted(shuffled_data, key=lambda x: x[0])
    chunks = create_chunks(shuffled_data)
    return chunks

def reduce_rating(values_to_reduce):
    result = {}
    for key, value in values_to_reduce:
        if key in result:
            result[key] += 1
        else:
            result[key] = 1
    return result

def map_shuffle(data, n_nodes):
    map_result = map_rating(data)
    shuffle_result = shuffle_rating(map_result, n_nodes)
    return shuffle_result

In [33]:
with Parallel(n_jobs=n_nodes, verbose=10) as parallel:
    map_shuffle_result = parallel(delayed(map_shuffle)(row, n_nodes) for row in data)

[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Batch computation too fast (0.0037870407104492188s.) Setting batch_size=2.
[Parallel(n_jobs=5)]: Done   3 tasks      | elapsed:    0.0s
[Parallel(n_jobs=5)]: Done   8 tasks      | elapsed:    0.0s
[Parallel(n_jobs=5)]: Batch computation too fast (0.007159233093261719s.) Setting batch_size=4.
[Parallel(n_jobs=5)]: Done  20 tasks      | elapsed:    0.0s
[Parallel(n_jobs=5)]: Batch computation too fast (0.005446910858154297s.) Setting batch_size=8.
[Parallel(n_jobs=5)]: Done  38 tasks      | elapsed:    0.0s
[Parallel(n_jobs=5)]: Done  78 tasks      | elapsed:    0.0s
[Parallel(n_jobs=5)]: Batch computation too fast (0.006251335144042969s.) Setting batch_size=16.
[Parallel(n_jobs=5)]: Done 150 tasks      | elapsed:    0.0s
[Parallel(n_jobs=5)]: Batch computation too fast (0.0051348209381103516s.) Setting batch_size=32.
[Parallel(n_jobs=5)]: Done 342 tasks      | elapsed:    0.0s
[Parallel(n_jo

In [34]:
shuffle_stage = {i: [] for i in range(n_nodes)}
for values in map_shuffle_result:
    values = dict(values)
    for key in values.keys():
        shuffle_stage[key].extend(values[key])

In [35]:
with Parallel(n_jobs=n_nodes, verbose=10) as parallel:
    reduce_result = parallel(delayed(reduce_rating)(shuffle_stage[key]) for key in shuffle_stage.keys())

[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Batch computation too fast (0.06485128402709961s.) Setting batch_size=2.
[Parallel(n_jobs=5)]: Done   2 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=5)]: Done   3 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed:    0.0s finished


После reduce все можно собрать в одном цикле, считаем, что данные переслали после на 1 машину и агрегируем

In [36]:
result = {}
for partition in reduce_result:
    for key in partition.keys():
        if key in result:
            result[key] += partition[key]
        else:
            result[key] = partition[key]

In [37]:
sorted(result.items(), key=lambda x: x[1], reverse=True)

[('2571', 900),
 ('318', 887),
 ('296', 878),
 ('2959', 828),
 ('4993', 756),
 ('7153', 719),
 ('5952', 697),
 ('858', 690),
 ('50', 688),
 ('2858', 680),
 ('4226', 636),
 ('260', 607),
 ('593', 606),
 ('1196', 601),
 ('4973', 594),
 ('1198', 588),
 ('356', 588),
 ('527', 572),
 ('7361', 567),
 ('1136', 554),
 ('47', 532),
 ('2762', 507),
 ('1193', 477),
 ('1221', 470),
 ('541', 462),
 ('2028', 452),
 ('608', 451),
 ('750', 446),
 ('6874', 446),
 ('32', 436),
 ('32587', 431),
 ('1089', 430),
 ('2329', 422),
 ('110', 413),
 ('8961', 412),
 ('4878', 410),
 ('3578', 409),
 ('1210', 408),
 ('1197', 408),
 ('1206', 407),
 ('912', 396),
 ('1270', 393),
 ('589', 391),
 ('1214', 389),
 ('1213', 387),
 ('6016', 383),
 ('7438', 380),
 ('1208', 378),
 ('6539', 377),
 ('4306', 377),
 ('6377', 374),
 ('1291', 372),
 ('293', 371),
 ('1258', 370),
 ('33794', 369),
 ('3996', 367),
 ('4995', 367),
 ('6711', 366),
 ('2997', 361),
 ('1704', 359),
 ('1265', 357),
 ('924', 352),
 ('4027', 343),
 ('1080', 3